<a href="https://colab.research.google.com/github/kridtapon/Momentum-Matrix/blob/main/Momentum_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.2 MB/s eta 0:00:00


In [58]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """
    Calculate MACD, Signal Line, and Histogram.
    """
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal_line

# Function to calculate Ichimoku Kijun-Sen (Baseline)
def calculate_kijun_sen(df, period=26):
    """
    Calculate Ichimoku Kijun-Sen (Baseline).
    """
    high_rolling = df['High'].rolling(window=period).max()
    low_rolling = df['Low'].rolling(window=period).min()
    kijun_sen = (high_rolling + low_rolling) / 2
    return kijun_sen

# Define the stock symbol and time period
symbol = 'NRG'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate MACD and Signal Line
df['MACD'], df['Signal_Line'] = calculate_macd(df, short_window=7, long_window=20, signal_window=7)

# Calculate Ichimoku Kijun-Sen
df['Kijun_Sen'] = calculate_kijun_sen(df, period=5)

# Define Entry and Exit signals
df['Entry'] = (df['MACD'] > df['Signal_Line']) & (df['Close'] > df['Kijun_Sen'])
df['Exit'] = (df['MACD'] < df['Signal_Line']) & (df['Close'] < df['Kijun_Sen'])

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                             365223.2188
Total Return [%]                       265.223219
Benchmark Return [%]                   175.103187
Max Gross Exposure [%]                      100.0
Total Fees Paid                      24373.048624
Max Drawdown [%]                        20.195723
Max Drawdown Duration                       189.0
Total Trades                                   56
Total Closed Trades                            56
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            51.785714
Best Trade [%]                          28.565154
Worst Trade [%]                        -13.278382
Avg Winning Trade [%]                    8.496957
Avg Losing Trade [%]                    -3.619033


In [57]:
import itertools
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal_line

# Function to calculate Ichimoku Kijun-Sen (Baseline)
def calculate_kijun_sen(df, period=26):
    high_rolling = df['High'].rolling(window=period).max()
    low_rolling = df['Low'].rolling(window=period).min()
    kijun_sen = (high_rolling + low_rolling) / 2
    return kijun_sen

# Define the stock symbol and time period
symbol = 'NRG'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Grid search parameter ranges
short_window_range = range(5, 20, 2)
long_window_range = range(20, 41, 2)
signal_window_range = range(5, 20, 2)
kijun_sen_period_range = range(5, 20, 2)

# Store results
results = []

# Iterate over all combinations of parameters
for short_window, long_window, signal_window, kijun_sen_period in itertools.product(
    short_window_range, long_window_range, signal_window_range, kijun_sen_period_range
):
    # Skip invalid parameter combinations
    if short_window >= long_window:
        continue

    # Calculate MACD and Signal Line
    df['MACD'], df['Signal_Line'] = calculate_macd(df, short_window, long_window, signal_window)

    # Calculate Ichimoku Kijun-Sen
    df['Kijun_Sen'] = calculate_kijun_sen(df, kijun_sen_period)

    # Define Entry and Exit signals
    df['Entry'] = (df['MACD'] > df['Signal_Line']) & (df['Close'] > df['Kijun_Sen'])
    df['Exit'] = (df['MACD'] < df['Signal_Line']) & (df['Close'] < df['Kijun_Sen'])

    # Filter data for the test period (2020-2025)
    test_df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=test_df['Close'],
        entries=test_df['Entry'],
        exits=test_df['Exit'],
        init_cash=100_000,
        fees=0.001
    )

    # Calculate performance metrics (e.g., final equity, Sharpe ratio)
    final_equity = portfolio.total_return()

    # Save results
    results.append((short_window, long_window, signal_window, kijun_sen_period, final_equity))

# Convert results to a DataFrame
results_df = pd.DataFrame(
    results,
    columns=['Short_Window', 'Long_Window', 'Signal_Window', 'Kijun_Sen_Period', 'Final_Equity']
)

# Find the best parameters based on final equity
best_params = results_df.loc[results_df['Final_Equity'].idxmax()]
print("Best Parameters:")
print(best_params)


[*********************100%***********************]  1 of 1 completed


Best Parameters:
Short_Window         7.000000
Long_Window         20.000000
Signal_Window        7.000000
Kijun_Sen_Period     5.000000
Final_Equity         2.652232
Name: 712, dtype: float64
